In [1]:
import subprocess

In [2]:
import pandas

In [3]:
import pickle

### TSV files

In [4]:
with open('chemprot_sample_abstracts.tsv') as openfile:
    f = pandas.read_csv(openfile,delimiter="\t", header=None, names=["docid", "title", "abstract"])
#     for line in openfile.readlines()[0]:
#         print line.strip().split("\t")

In [5]:
with open('chemprot_sample_abstracts.tsv') as openfile:
    print openfile.readline().strip("\n").split("\t")
    (docid,_,_) = openfile.readline().strip("\n").split("\t")
    
    

['10471277', 'Probing the salmeterol binding site on the beta 2-adrenergic receptor using a novel photoaffinity ligand, [(125)I]iodoazidosalmeterol.', 'Salmeterol is a long-acting beta2-adrenergic receptor (beta 2AR) agonist used clinically to treat asthma. In addition to binding at the active agonist site, it has been proposed that salmeterol also binds with very high affinity at a second site, termed the "exosite", and that this exosite contributes to the long duration of action of salmeterol. To determine the position of the phenyl ring of the aralkyloxyalkyl side chain of salmeterol in the beta 2AR binding site, we designed and synthesized the agonist photoaffinity label [(125)I]iodoazidosalmeterol ([125I]IAS). In direct adenylyl cyclase activation, in effects on adenylyl cyclase after pretreatment of intact cells, and in guinea pig tracheal relaxation assays, IAS and the parent drug salmeterol behave essentially the same. Significantly, the photoreactive azide of IAS is positioned

In [6]:
for i in f:
    print i

docid
title
abstract


### Use LeadMine to extract NER from (Multiple) txt files

In [17]:
import os

In [33]:
leadmine_jar_path = '/home/antonis/LeadMine/leadmine-3.11.jar'
docs_path = '/home/antonis/data/patents/EP/EP__11_Training_relevancy_relations_corpus/'
leadmine_output_path = '/home/antonis/helpers/EP_11_LeadMined.tsv'


with open(output_path, 'wb') as f:
    subprocess.call(['java', '-jar', leadmine_jar_path, '-tsv', docs_path, ' > ', output_path], stdout=f)

### Read entities from file & process them in snorkel input format (DefaultDict)

In [15]:
# TODO: put it on a py helper file.

In [17]:
# from collections import defaultdict
# def entities_to_dict(entities, doc_col= 'doc_id', type_col = 'EntityType', name_col = 'EntityText', start_col = 'start', stop_col = 'stop'):
#     '''From a dataframe to unary tags dictionary'''
#     entities["tup"] = map(lambda x,y,start,end: ("|".join([x,y]),start,end) , entities[type_col], entities[name_col], entities[start_col],entities[stop_col])
#     unary_tags = df[[doc_col,'tup']].set_index(doc_col).stack().groupby(doc_col).apply(set).to_dict()
#     return defaultdict(set,unary_tags)

In [4]:
from preprocessing import entities_to_dict

In [5]:
leadmine_output_path = '/home/antonis/helpers/EP_11_LeadMined.tsv'
df = pandas.read_table(leadmine_output_path)
unary_tags = entities_to_dict(df, start_col='BegIndex', stop_col='EndIndex', doc_col='DocName')

### Write to bz2 pickle

In [84]:
import bz2
import cPickle as pickle
with bz2.BZ2File('EP_11_LeadMined_unary_tags.pkl.bz2', 'w') as f:
    pickle.dump(unary_tags, f)

#### # # # # # # # # # # # # # 

### Old code for df -> pkl

In [62]:
entities = pandas.read_table(output_path).drop(['PossiblyCorrectedText','SectionType',"CorrectionDistance", 'ResolvedForm'], axis=1)

In [63]:
entities["tup"] = map(lambda x,y,start,end: ("|".join([x,y]),start,end) , entities['EntityType'], entities['EntityText'], entities['BegIndex'],entities['EndIndex'])

In [50]:
entities.drop(['EntityType','EntityText'], axis=1, inplace=True)

In [66]:
entities.head()

,DocName,BegIndex,EndIndex,EntityType,EntityText,tup
0,EP2460782B1_abstract_000,81,93,M,cerium oxide,"(M|cerium oxide, 81, 93)"
1,EP2266989B1_claims_001,100,106,E,carbon,"(E|carbon, 100, 106)"
2,EP2266989B1_claims_001,110,118,E,nitrogen,"(E|nitrogen, 110, 118)"
3,EP2266989B1_claims_001,167,175,E,nitrogen,"(E|nitrogen, 167, 175)"
4,EP2266989B1_claims_001,289,297,E,hydrogen,"(E|hydrogen, 289, 297)"


In [75]:
unary_tags = entities[['DocName','tup']].set_index('DocName').stack().groupby('DocName').apply(set).to_dict()

In [79]:
#convert to defaultdict - dunno if needed
unary_tags = defaultdict(set,unary_tags)

In [80]:
unary_tags

defaultdict(set,
            {'EP2266989B1_claims_000': {('A|amino', 482, 487),
              ('A|amino', 1785, 1790),
              ('A|amino', 2796, 2801),
              ('A|amino', 3833, 3838),
              ('A|amino', 4855, 4860),
              ('A|amino', 6048, 6053),
              ('A|amino', 10041, 10046),
              ('A|amino', 10736, 10741),
              ('A|amino', 11633, 11638),
              ('A|hydroxy', 330, 337),
              ('A|hydroxy', 1411, 1418),
              ('A|hydroxy', 2589, 2596),
              ('A|hydroxy', 3097, 3104),
              ('A|hydroxy', 3189, 3196),
              ('A|hydroxy', 3715, 3722),
              ('A|hydroxy', 4753, 4760),
              ('A|hydroxy', 5156, 5163),
              ('A|hydroxy', 5248, 5255),
              ('A|hydroxy', 5917, 5924),
              ('A|hydroxy', 9212, 9219),
              ('A|hydroxy', 9414, 9421),
              ('A|hydroxy', 9506, 9513),
              ('A|hydroxy', 9947, 9954),
              ('A|hydroxy', 10

In [81]:
unary_tags['EP2266989B1_claims_000']

{('A|amino', 482, 487),
 ('A|amino', 1785, 1790),
 ('A|amino', 2796, 2801),
 ('A|amino', 3833, 3838),
 ('A|amino', 4855, 4860),
 ('A|amino', 6048, 6053),
 ('A|amino', 10041, 10046),
 ('A|amino', 10736, 10741),
 ('A|amino', 11633, 11638),
 ('A|hydroxy', 330, 337),
 ('A|hydroxy', 1411, 1418),
 ('A|hydroxy', 2589, 2596),
 ('A|hydroxy', 3097, 3104),
 ('A|hydroxy', 3189, 3196),
 ('A|hydroxy', 3715, 3722),
 ('A|hydroxy', 4753, 4760),
 ('A|hydroxy', 5156, 5163),
 ('A|hydroxy', 5248, 5255),
 ('A|hydroxy', 5917, 5924),
 ('A|hydroxy', 9212, 9219),
 ('A|hydroxy', 9414, 9421),
 ('A|hydroxy', 9506, 9513),
 ('A|hydroxy', 9947, 9954),
 ('A|hydroxy', 10626, 10633),
 ('A|hydroxy', 11539, 11546),
 ('A|hydroxy', 11755, 11762),
 ('A|hydroxy', 11847, 11854),
 ('A|thioxo', 1922, 1928),
 ('A|thioxo', 2045, 2051),
 ('A|thioxo', 2179, 2185),
 ('A|thioxo', 3452, 3458),
 ('A|thioxo', 3970, 3976),
 ('A|thioxo', 4093, 4099),
 ('A|thioxo', 11042, 11048),
 ('E|carbon', 130, 136),
 ('E|carbon', 1848, 1854),
 ('E|carb